In [1]:
import numpy as np
import xarray as xr
import dask
import os
from glob import glob

### Preliminaries

In [2]:
###############################
# Set paths
# UPDATE THIS FOR REPRODUCTION
###############################
in_path = '/gpfs/group/kaf26/default/rls66/GMFD_25/'

In [3]:
############
# Dask
############
from dask_jobqueue import PBSCluster
cluster = PBSCluster(cores=1, resource_spec = 'pmem=15GB', memory='15GB',
                     worker_extra_args= ['#PBS -l feature=rhel7'], walltime = '00:10:00')

cluster.scale(jobs=20)  # ask for jobs

from dask.distributed import Client
client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.102.201.236:39144,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Temperature

In [4]:
# Read all
ds_tmin = xr.open_mfdataset(in_path + 'tmin_daily_*', parallel=True, chunks='auto')
ds_tmax = xr.open_mfdataset(in_path + 'tmax_daily_*', parallel=True, chunks='auto')

# Select time period
ds_tmin = ds_tmin.sel(time=slice('1980-01-01','2014-12-31'))
ds_tmax = ds_tmax.sel(time=slice('1980-01-01','2014-12-31'))

# Get tasavg
ds_tas = (ds_tmin['tmin'] + ds_tmax['tmax']) / 2.

In [5]:
# Merge
ds = xr.merge([xr.Dataset({'tas':ds_tas}), ds_tmin, ds_tmax])

In [6]:
%%time
# rechunk for quantile calculations
ds = ds.chunk({'time':-1, 'lat':50, 'lon':50})

# Get 99.95th, 99.97th, 99.99th quantiles
ds_rp5 = ds.quantile(q=0.9995, dim='time').compute() # approx 1-in-5 year event
ds_rp10 = ds.quantile(q=0.9997, dim='time').compute() # approx 1-in-10 year event
ds_rp20 = ds.quantile(q=0.9999, dim='time').compute() # approx 1-in-20 year event

CPU times: user 5min 30s, sys: 13.2 s, total: 5min 43s
Wall time: 12min 30s


In [8]:
# Merge and store
ds_out = xr.merge([ds_rp5.drop('quantile').rename({'tas':'tas_rp5',
                                                   'tmax':'tasmax_rp5',
                                                   'tmin':'tasmin_rp5'}),
                   ds_rp10.drop('quantile').rename({'tas':'tas_rp10',
                                                   'tmax':'tasmax_rp10',
                                                   'tmin':'tasmin_rp10'}),
                   ds_rp20.drop('quantile').rename({'tas':'tas_rp20',
                                                    'tmax':'tasmax_rp20',
                                                    'tmin':'tasmin_rp20'})])

ds_out -= 273.15 # K -> C

ds_out['lon'] = np.where(ds_out['lon'] > 180, ds_out['lon'] - 360, ds_out['lon']) # update lon
ds_out = ds_out.sortby('lon')

ds_out.to_netcdf('../data/gmfd_temperature_quantiles.nc')

# Precipitation

In [4]:
# Read 
ds = xr.open_mfdataset(in_path + 'prcp_daily_*', parallel=True, chunks='auto')

# Select time period
ds = ds.sel(time=slice('1980-01-01','2014-12-31'))

In [5]:
%%time
# rechunk for quantile calculations
ds = ds.chunk({'time':-1, 'lat':50, 'lon':50})

# Get 99.95th, 99.97th, 99.99th quantiles (wet days only: >1mm)
ds_rp5 = ds.where(ds.prcp > 1./86400).quantile(q=0.9995, dim='time').compute() # approx 1-in-5 year event
ds_rp10 = ds.where(ds.prcp > 1./86400).quantile(q=0.9997, dim='time').compute() # approx 1-in-10 year event
ds_rp20 = ds.where(ds.prcp > 1./86400).quantile(q=0.9999, dim='time').compute() # approx 1-in-20 year event

CPU times: user 1min 56s, sys: 4.56 s, total: 2min 1s
Wall time: 4min 32s


In [6]:
# Merge and store
ds_out = xr.merge([ds_rp5.drop('quantile').rename({'prcp':'pr_rp5'}),
                   ds_rp10.drop('quantile').rename({'prcp':'pr_rp10'}),
                   ds_rp20.drop('quantile').rename({'prcp':'pr_rp20'})])

ds_out *= 86400
ds_out.to_netcdf('../data/gmfd_precip_quantiles.nc')